# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
import time
from sklearn.cross_validation import train_test_split
from datetime import datetime

tf.logging.set_verbosity(tf.logging.INFO)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data loading

In [2]:
width = 640
height = 480
channel = 3
total_pix = width * height * channel
num_class = 2

In [3]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out

In [4]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]))

1780


In [5]:
tr_data_dir = data_process(data_dir1, 0.2)

In [6]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    tr_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    tr_label[i] = tr_data_dir['train'][i][1]
    
    

    
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    ev_data[i][:] = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC).flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [ ]:
tr_label = np.zeros((len(tr_data_dir['train']), 2), dtype=np.float32)

for i in range(len(tr_data_dir['train'])):
    if tr_data_dir['train'][i][1] == 0:
        tr_label[i][0] = 1
        tr_label[i][1] = 0
    
    else:
        tr_label[i][0] = 0
        tr_label[i][1] = 1
    
    

ev_label = np.zeros((len(tr_data_dir['test']), 2), dtype=np.float32)

for i in range(len(tr_data_dir['test'])):
    if tr_data_dir['test'][i][1] == 0:
        ev_label[i][0] = 1
        ev_label[i][1] = 0
        
    else:
        ev_label[i][0] = 0
        ev_label[i][1] = 1

# CNN model

In [ ]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

In [ ]:
def conv2d(x, W, n):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=n)

def max_pool_2x2(x, n):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME', name = n)

In [7]:
X = tf.placeholder("float", shape = [None, total_pix], name='X')

# float32 - for sigmoid cross entropy
# int64 - for sparse softmax cross entropy
Y = tf.placeholder(tf.int64, shape = [None, 1], name='Y')
#Y = tf.placeholder("float", shape = [None, 2], name='Y')

keep_prob = tf.placeholder(tf.float32, name='keep_p')

In [8]:
"""Model function for CNN."""

input_layer = tf.reshape(X, [-1, 480, 640, 3])


conv1 = tf.layers.conv2d(

    inputs=input_layer,
    filters=48,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = 'conv1')

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')



conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=96,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = 'conv2')

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')



conv3 = tf.layers.conv2d(
    inputs=pool2,
    filters=96,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = 'conv3')

pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, name='pool3')



conv4 = tf.layers.conv2d(
    inputs=pool3,
    filters=96,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = 'conv4')

pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2, name='pool4')



conv5 = tf.layers.conv2d(
    inputs=pool4,
    filters=96,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = 'conv5')

pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2, name='pool5')
pool5_flat = tf.reshape(pool5, [-1, 15 * 20 * 96])

dense = tf.layers.dense(inputs=pool5_flat, units=1024, activation=tf.nn.relu, name='fc1')

dropout = tf.layers.dropout(inputs=dense, rate=(1-keep_prob), name='fc_drop')


logits = tf.layers.dense(inputs=dropout, units=2, name='logits')
y_conv = tf.nn.softmax(logits, name='softmax_tensor')



In [ ]:
x_img = tf.reshape(X, [-1, height, width, channel])

wc1 = weight_variable([5, 5, 3, 48], 'W1')
bc1 = bias_variable([48], 'b1')
hc1 = tf.nn.relu(conv2d(x_img, wc1, 'conv1') + bc1)
hp1 = max_pool_2x2(hc1, 'max1')

wc2 = weight_variable([5, 5, 48, 96], 'W2')
bc2 = bias_variable([96], 'b2')
hc2 = tf.nn.relu(conv2d(hp1, wc2, 'conv2') + bc2)
hp2 = max_pool_2x2(hc2, 'max2')

wc3 = weight_variable([5, 5, 96, 96], 'W3')
bc3 = bias_variable([96], 'b3')
hc3 = tf.nn.relu(conv2d(hp2, wc3, 'conv3') + bc3)
hp3 = max_pool_2x2(hc3, 'max3')

wc4 = weight_variable([5, 5, 96, 96], 'W4')
bc4 = bias_variable([96], 'b4')
hc4 = tf.nn.relu(conv2d(hp3, wc4, 'conv4') + bc4)
hp4 = max_pool_2x2(hc4, 'max4')

wc5 = weight_variable([5, 5, 96, 96], 'W5')
bc5 = bias_variable([96], 'b5')
hc5 = tf.nn.relu(conv2d(hp4, wc5, 'conv5') + bc5)
hp5 = max_pool_2x2(hc5, 'max5')

wf1 = weight_variable([15*20*96, 1024], 'Wf1')
bf1 = bias_variable([1024], 'bf1')
hf_flat = tf.reshape(hp5, [-1, 15*20*96])
hf1 = tf.nn.relu(tf.matmul(hf_flat, wf1) + bf1)
hf_drop = tf.nn.dropout(hf1, keep_prob)

wf2 = weight_variable([1024, num_class], 'Wf2')
bf2 = bias_variable([num_class], 'bf2')

logits = tf.matmul(hf_drop, wf2) + bf2
y_conv = tf.nn.softmax(logits)



# Training

In [9]:
today = datetime.today().strftime("%m-%d-%H-%M")
print(today)

05-11-05-55


In [10]:
learning_r = 1.0
batch_s = 16
epochs = 5
inner_steps = int((tr_data.shape[0])/batch_s)
total_steps = int((tr_data.shape[0]/batch_s*epochs))
print(total_steps, inner_steps)
disp_step = 10

816 163


In [11]:
class_out = tf.argmax(input=logits, axis=1, name='argmax')
loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
#loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y)

train_step = tf.train.GradientDescentOptimizer(learning_rate = learning_r).minimize(loss=loss,
                                                                                    global_step=tf.train.get_global_step())
correct_prediction = tf.equal(tf.argmax(y_conv, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
loss = tf.losses.sparse_softmax_cross_entropy(labels = Y, logits = logits)
train_step = tf.train.GradientDescentOptimizer(learning_rate = learning_r).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
ev_accuracy = np.zeros((len(ev_label), 1))

In [15]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [16]:
t_count = 1

for j in range(epochs):

    for i in range(inner_steps):

        tr_batch = {}

        if i < inner_steps-1:
            tr_batch['data'] = tr_data[i*batch_s:(i+1)*batch_s][:]
            tr_batch['label'] = tr_label[i*batch_s:(i+1)*batch_s][:]

        else:
            tr_batch['data'] = tr_data[i*batch_s:][:]
            tr_batch['label'] = tr_label[i*batch_s:][:]
            
        sess.run(train_step, feed_dict={X: tr_batch['data'],
                                        Y: tr_batch['label'],
                                        keep_prob: 0.5})


        if t_count % disp_step == 0:
            
            loss_tr = sess.run(loss, feed_dict = {X: tr_batch['data'],
                                                  Y: tr_batch['label'],
                                                  keep_prob: 1.0})
            
            acc_tr = sess.run(accuracy, feed_dict = {X: tr_batch['data'],
                                                     Y: tr_batch['label'],
                                                     keep_prob: 1.0})
            
            print('step %d, training accuracy %g, loss %f' % (t_count, acc_tr, loss_tr))
            #print(acc_tr, loss_tr)

        t_count = t_count+1


for k in range(len(ev_label)):
    ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_data[k:k+1],
                                                   Y: ev_label[k:k+1],
                                                   keep_prob: 1.0})

step 10, training accuracy 0.5, loss 0.695179
step 20, training accuracy 0.4375, loss 0.695474
step 30, training accuracy 0.5, loss 0.701901
step 40, training accuracy 0.625, loss 0.683379
step 50, training accuracy 0.5625, loss 0.685385
step 60, training accuracy 0.5, loss 0.693409
step 70, training accuracy 0.625, loss 0.669257
step 80, training accuracy 0.5625, loss 0.685469
step 90, training accuracy 0.5625, loss 0.685408
step 100, training accuracy 0.5, loss 0.697416
step 110, training accuracy 0.625, loss 0.669756
step 120, training accuracy 0.4375, loss 0.699180
step 130, training accuracy 0.5625, loss 0.685326
step 140, training accuracy 0.625, loss 0.663885
step 150, training accuracy 0.5625, loss 0.692347
step 160, training accuracy 0.5625, loss 0.687207
step 170, training accuracy 0.625, loss 0.672104
step 180, training accuracy 0.6875, loss 0.629885
step 190, training accuracy 0.5, loss 0.731438
step 200, training accuracy 0.5625, loss 0.685706
step 210, training accuracy 0

In [ ]:
saver = tf.train.saver()
save_path = '/model/CNN2600-Vis'
saver.save(sess, save_path)

In [ ]:
print(np.mean(ev_accuracy))

In [ ]:
test1 = np.where(ev_label == 0)
test2 = np.where(ev_label == 1)

In [ ]:
print(len(test1[0]), len(test2[0]))

In [ ]:
print(356/(356+297))

In [ ]:
print(tr_batch['label'])